# Optimus Mining Algo

In [1]:
from kaggle_environments import evaluate, make
from kaggle_environments.envs.halite.helpers import *
import copy
import numpy as np
import random
import scipy.optimize
from board_viz import draw_game, draw_board
import math

board_size = 21

# Optimus Miner

In [2]:
ship_target={}   # where to go to collect
all_actions=[ShipAction.NORTH, ShipAction.EAST,ShipAction.SOUTH,ShipAction.WEST]
num_shipyard_targets = 1
turns_optimal=np.array(
  [[0, 2, 3, 4, 4, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 8],
   [0, 1, 2, 3, 3, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7, 7, 7],
   [0, 0, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7],
   [0, 0, 1, 2, 2, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6],
   [0, 0, 0, 1, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6],
   [0, 0, 0, 0, 0, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5],
   [0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

def limit(x,a,b):
  if x<a:
    return a
  if x>b:
    return b
  return x

def num_turns_to_mine(C,H,rt_travel):
  #How many turns should we plan on mining?
  #C=carried halite, H=halite in square, rt_travel=steps to square and back to shipyard
  if C==0:
    ch=0
  elif H==0:
    ch=turns_optimal.shape[0]
  else:
    ch=int(math.log(C/H)*2.5+5.5)
    ch=limit(ch,0,turns_optimal.shape[0]-1)
  rt_travel=int(limit(rt_travel,0,turns_optimal.shape[1]-1))
  return turns_optimal[ch,rt_travel]

def halite_per_turn(carrying, halite,travel,min_mine=1):
  #compute return from going to a cell containing halite, using optimal number of mining steps
  #returns halite mined per turn, optimal number of mining steps
  #Turns could be zero, meaning it should just return to a shipyard (subject to min_mine)
  turns=num_turns_to_mine(carrying,halite,travel)
  if turns<min_mine:
    turns=min_mine
  mined=carrying+(1-.75**turns)*halite
  return mined/(travel+turns), turns

def nearest_shipyard(pos, player):
  #return distance, position of nearest shipyard to pos.  100,None if no shipyards
  mn=30
  best_pos=None
  for sy in player.shipyards:
    d=dist(pos, sy.position)
    if d<mn:
      mn=d
      best_pos=sy.position
  return mn,best_pos

def dirs_to(p1, p2, size=21):
  #Get the actions you should take to go from Point p1 to Point p2
  #using shortest direction by wraparound
  #Args: p1: from Point
  #      p2: to Point
  #      size:  size of board
  #returns: list of directions, tuple (deltaX,deltaY)
  #The list is of length 1 or 2 giving possible directions to go, e.g.
  #to go North-East, it would return [ShipAction.NORTH, ShipAction.EAST], because
  #you could use either of those first to go North-East.
  #[None] is returned if p1==p2 and there is no need to move at all
  deltaX, deltaY=p2 - p1
  if abs(deltaX)>size/2:
    #we wrap around
    if deltaX<0:
      deltaX+=size
    elif deltaX>0:
      deltaX-=size
  if abs(deltaY)>size/2:
    #we wrap around
    if deltaY<0:
      deltaY+=size
    elif deltaY>0:
      deltaY-=size
  #the delta is (deltaX,deltaY)
  ret=[]
  if deltaX>0:
    ret.append(ShipAction.EAST)
  if deltaX<0:
    ret.append(ShipAction.WEST)
  if deltaY>0:
    ret.append(ShipAction.NORTH)
  if deltaY<0:
    ret.append(ShipAction.SOUTH)
  if len(ret)==0:
    ret=[None]  # do not need to move at all
    
  if len(ret)>1:
    random.shuffle(ret)
    

  return ret, (deltaX,deltaY)

def dist(a,b):
  #Manhattan distance of the Point difference a to b, considering wrap around
  action,step=dirs_to(a, b, size=21) 
  return abs(step[0]) + abs(step[1])


def assign_targets(board, ships, player):
  #Assign the ships to a cell containing halite optimally
  #set ship_target[ship_id] to a Position
  #We assume that we mine halite containing cells optimally or return to deposit
  #directly if that is optimal, based on maximizing halite per step.
  #Make a list of pts containing cells we care about, this will be our columns of matrix C
  #the rows are for each ship in collect
  #computes global dict ship_tagert with shipid->Position for its target
  #global ship targets should already exist
  old_target=copy.copy(ship_target)
  ship_target.clear()
  if len(ships)==0:
    return
  halite_min=50
  pts1=[]
  pts2=[]
  for pt,c in board.cells.items():
    assert isinstance(pt,Point)
    if c.halite > halite_min:
      pts1.append(pt)
  #Now add duplicates for each shipyard - this is directly going to deposit
  for sy in player.shipyards:
    for i in range(num_shipyard_targets):
      pts2.append(sy.position)
  #this will be the value of assigning C[ship,pt]
  C=np.zeros((len(ships),len(pts1)+len(pts2)))
  #this will be the optimal mining steps we calculated
  for i,ship in enumerate(ships):
    for j,pt in enumerate(pts1+pts2):
      #two distances: from ship to halite, from halite to nearest shipyard
      d1=dist(ship.position,pt)
      d2,shipyard_position=nearest_shipyard(pt, player)
      if shipyard_position is None:
        #don't know where to go if no shipyard
        d2=1
      #value of target is based on the amount of halite per turn we can do
      my_halite=ship.halite
      if j < len(pts1):
        #in the mining section
        v, mining=halite_per_turn(my_halite,board.cells[pt].halite, d1+d2)
        #mining is no longer 0, due to min_mine (default)
      else:
        #in the direct to shipyard section
        if d1>0:
          v=my_halite/d1
        else:
          #we are at a shipyard
          v=0
      if board.cells[pt].ship and board.cells[pt].ship.player_id != player.id:
        #if someone else on the cell, see how much halite they have
        #enemy ship
        enemy_halite=board.cells[pt].ship.halite
        if enemy_halite <= my_halite:
          v = -1000   # don't want to go there
        else:
          if d1 < 3:
            #attack or scare off if reasonably quick to get there
            v += enemy_halite/(d1+1)  # want to attack them or scare them off
      #print('shipid {} col {} is {} with {:8.1f} score {:8.2f}'.format(ship.id,j, pt,board.cells[pt].halite,v))
      C[i, j] = v
  #Compute the optimal assignment
  row,col=scipy.optimize.linear_sum_assignment(C, maximize=True)
  #so ship row[i] is assigned to target col[j]
  #print('got row {} col {}'.format(row,col))
  #print(C[row[0],col[0]])
  pts = pts1 + pts2
  for r, c in zip(row, col):
    ship_target[ships[r].id] = pts[c]
  for id, t in ship_target.items():
    st = ''
    ta = ''
    if board.ships[id].position==t:
      st='MINE'
    elif len(player.shipyards)>0 and t==player.shipyards[0].position:
      st='SHIPYARD'
    if id not in old_target or old_target[id] != ship_target[id]:
      ta=' NEWTARGET'
  get_ship_actions(ships)
  return

def get_ship_actions(ships):
    actions={}   # record actions for each ship
    for ship in ships:
        if ship.id in ship_target:
            a,delta = dirs_to(ship.position, ship_target[ship.id], size=21)
            actions[ship.id]=a
        else:
            actions[ship.id]=[random.choice(all_actions)]
    ship.next_action = actions[ship.id][0]

In [3]:
def setup_board():
    environment = make("halite")
    agent_count = 4
    environment.reset(agent_count)
    state = environment.state[0]
    return Board(state.observation, environment.configuration)

def convert_and_spawn(board):
    for ship in board.ships.values():
        ship.next_action = ShipAction.CONVERT
    board = board.next()
    for shipyard in board.shipyards.values():
        shipyard.next_action = ShipyardAction.SPAWN
    return board.next()


In [4]:
from mining_search import find_best_paths, mining_locs, path_to_action_list
import time

def get_p0_action_list(board, t_max, board_size=21):
    p0_start_pos = board.players[0].ships[0].position
    p0_solutions = find_best_paths(board, board_size, t_max, [p0_start_pos], time.time() + 1.0)
    p0_solution = p0_solutions[p0_start_pos.x, p0_start_pos.y].path
    return path_to_action_list(p0_solution)

def test_case(t_max, draw_this_game=False):
    ship_target = {}
    boards = []
    board = setup_board()
    board = convert_and_spawn(board)
    boards.append(board)
    p0_actions = get_p0_action_list(board, t_max)
    for t in range(t_max):
        for ship in board.players[0].ships:
            ship.next_action = p0_actions[t]
        assign_targets(board, board.players[3].ships, board.players[3])
        board = board.next()
        boards.append(board)
    if draw_this_game:
        draw_game(boards)
    return (board.players[0].halite -4000, board.players[3].halite-4000)



In [5]:
def compare_mining(num_turns):
    sum_scores = 0, 0
    num_tests = 20
    for i in range(num_tests):
        scores = test_case(num_turns)
        sum_scores = sum_scores[0]+scores[0],sum_scores[1]+scores[1] 
    print(f'avg halite/turn for {num_turns} turns of mining')
    print('Djikstra:', sum_scores[0]/num_tests/num_turns, 'Optimus:', sum_scores[1]/num_tests/num_turns)
test_sizes = [10,20,30]
for size in test_sizes:
    compare_mining(size)

avg halite/turn for 10 turns of mining
Djikstra: 26.225 Optimus: 12.595
avg halite/turn for 20 turns of mining
Djikstra: 27.682499999999997 Optimus: 15.875
avg halite/turn for 30 turns of mining
Djikstra: 31.706666666666667 Optimus: 18.761666666666667


In [13]:
test_case(17, True)

interactive(children=(IntSlider(value=0, description='t', max=17), Checkbox(value=True, description='cell_hali…

(500, 348)